# Visualizing Label Locations

Purpose of this notebook is to visualize the dataset tile locations on an interactive map. Run this notebook using `floodmaps-sampling` environment.

In [1]:
from pathlib import Path
import re
from netCDF4 import Dataset

In [2]:
# Absolute path to sampling folder
SAMPLING_BASE_DIR = Path("/lcrc/project/hydrosm/dma/data")
prism_file = SAMPLING_BASE_DIR / "supplementary" / "prism" / "prismprecip_20160801_20241130.nc"
texas_labels = SAMPLING_BASE_DIR / "labels" / "texas"
illinois_labels = SAMPLING_BASE_DIR / "labels" / "illinois"
other_labels = SAMPLING_BASE_DIR / "labels" / "others"

### PRISM shapefile

In [3]:
# read PRISM data
with Dataset(prism_file, "r", format="NETCDF4") as nc:
    geotransform = nc["geotransform"][:]

def get_bounding_box(x, y, geotransform):
    upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size = geotransform
    minx = x * x_size + upper_left_x
    miny = (y + 1) * y_size + upper_left_y
    maxx = (x + 1) * x_size + upper_left_x
    maxy = y * y_size + upper_left_y

    return (minx, miny, maxx, maxy)

def get_centroid(minx, miny, maxx, maxy):
    return ((minx + maxx) / 2, (miny + maxy) / 2)

### Initialize Folium Map

In [4]:
import folium

# Use PRISM CRS EPSG 4269
m = folium.Map(location=(31.0624, -89.7708))

### Texas Dataset

In [5]:
texas_group = folium.FeatureGroup("Texas Dataset").add_to(m)
p = re.compile('label_(\d+)_(\d+)_(\d+)_(\d+).tif')
for label_path in texas_labels.glob('label_*.tif'):
    match = p.match(label_path.name)
    if match:
        y = int(match.group(3))
        x = int(match.group(4))
    else:
        raise ValueError(f'{label_path.name} does not match pattern')

    bbox = get_bounding_box(x, y, geotransform)
    minx, miny, maxx, maxy = bbox
    centroid = get_centroid(minx, miny, maxx, maxy)[::-1]

    folium.Rectangle(
        bounds=[(miny, minx), (maxy, maxx)],
        color="blue",
        weight=2,
        fill_color="lightblue",
        fill_opacity=0.5,
        fill=True
    ).add_to(texas_group)

    folium.Marker(
        location=centroid,
        icon=folium.Icon("blue"),
        popup=f"{label_path.name}").add_to(texas_group)

### Illinois Dataset

In [6]:
illinois_group = folium.FeatureGroup("Illinois Dataset").add_to(m)
p = re.compile('label_(\d+)_(\d+)_(\d+)_(\d+).tif')
for label_path in illinois_labels.glob('label_*.tif'):
    match = p.match(label_path.name)
    if match:
        y = int(match.group(3))
        x = int(match.group(4))
    else:
        raise ValueError(f'{label_path.name} does not match pattern')

    bbox = get_bounding_box(x, y, geotransform)
    minx, miny, maxx, maxy = bbox
    centroid = get_centroid(minx, miny, maxx, maxy)[::-1]

    folium.Rectangle(
        bounds=[(miny, minx), (maxy, maxx)],
        color="red",
        weight=2,
        fill_color="lightred",
        fill_opacity=0.5,
        fill=True
    ).add_to(illinois_group)

    folium.Marker(
        location=centroid,
        icon=folium.Icon("red"),
        popup=f"{label_path.name}").add_to(illinois_group)

In [60]:
folium.LayerControl().add_to(m)

## Others Dataset

In [8]:
others_group = folium.FeatureGroup("Others Dataset").add_to(m)
p = re.compile('label_(\d+)_(\d+)_(\d+)_(\d+).tif')
for label_path in other_labels.glob('label_*.tif'):
    match = p.match(label_path.name)
    if match:
        y = int(match.group(3))
        x = int(match.group(4))
    else:
        raise ValueError(f'{label_path.name} does not match pattern')

    bbox = get_bounding_box(x, y, geotransform)
    minx, miny, maxx, maxy = bbox
    centroid = get_centroid(minx, miny, maxx, maxy)[::-1]

    folium.Rectangle(
        bounds=[(miny, minx), (maxy, maxx)],
        color="green",
        weight=2,
        fill_color="lightgreen",
        fill_opacity=0.5,
        fill=True
    ).add_to(others_group)

    folium.Marker(
        location=centroid,
        icon=folium.Icon("green"),
        popup=f"{label_path.name}").add_to(others_group)

In [9]:
folium.LayerControl().add_to(m)

### Visualize in Interactive Map!

In [10]:
m